In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
# 提取申请时间
label = pd.read_csv('./data_raw/APP_5K_20171120_ADD_UGID.csv')
# label = label.drop(['month','user_gid','last_decision_dt','SamplingWeight'],axis=1).rename(columns={'create_time':'last_decision_tm'})
label = label[['ugid','create_time']].rename(columns={'create_time':'last_decision_tm'})

In [3]:
idcard01 = pd.read_csv("./data_mid/20171122_data_idcard01.201706_201708.csv")
idcard01 = pd.merge(idcard01.drop('utime', axis=1).drop_duplicates(), label, how='left', on='ugid')

In [4]:
%%time
idcard01['last_decision_tm'] = idcard01['last_decision_tm'].replace('DEC', '-12-',regex=True).\
replace('NOV', '-11-',regex=True).\
replace('OCT', '-10-',regex=True).\
replace('SEP', '-09-',regex=True).\
replace('AUG', '-08-',regex=True).\
replace('JUL', '-07-',regex=True).\
replace('JUN', '-06-',regex=True).\
replace('MAY', '-05-',regex=True).\
replace('APR', '-04-',regex=True).\
replace('MAR', '-03-',regex=True).\
replace('FEB', '-02-',regex=True).\
replace('JAN', '-01-',regex=True)

idcard01['last_decision_tm'] = idcard01['last_decision_tm'].apply(lambda x :datetime.strptime(x, '%d-%m-%Y:%H:%M:%S.%f') if x==x else None)

CPU times: user 112 ms, sys: 0 ns, total: 112 ms
Wall time: 110 ms


In [5]:
idcard01['month'] = idcard01['last_decision_tm'].apply(lambda x : x.month)

In [6]:
%%time
idcard01['ctime_len'] = idcard01['ctime'].apply(lambda x : len(str(int(round(float(x),0)))))

idcard01['ctime_stamp'] = idcard01.apply(lambda x : datetime.fromtimestamp(x['ctime']) if x['ctime_len']>8 else datetime.fromtimestamp(x['ctime']*1000), axis=1)

idcard01 = idcard01[idcard01.last_decision_tm == idcard01.last_decision_tm]

idcard01 = idcard01[idcard01.ctime_stamp <= idcard01.last_decision_tm]

CPU times: user 364 ms, sys: 0 ns, total: 364 ms
Wall time: 365 ms


In [7]:
idcard01.head()

,ugid,zuid,appid,ctime,address,allTime,birthday,edit_address_time,edit_birthday_time,edit_id_card_number_time,...,race,remote_addr,statistic_back_idcard_recognition_count,statistic_face_recognition_count,statistic_front_idcard_recognition_count,valid_date,last_decision_tm,month,ctime_len,ctime_stamp
0,e28c521f-63c2-40df-bdc6-94bd5915392d,f9c16c83189d4258a4326485689ca0ef|123456789001,jkios,1.497092e+06,四川省江安县怡乐镇凉水村大竹林组,8,1984年10月5日,0,0,0,...,汉,183.225.8.161,1,3.0,1,2009.02.02-2019.02.02,2017-08-11 21:26:11.213,8,7,2017-06-10 03:53:01.000
1,b6411ab1-80d5-484e-9018-a2018140a887,129c2f8d5c834e63b1b8e4fcca853a59|123456789001,jkioslunar,1.497163e+06,辽宁省昌图县铁南路粮库小区3号楼1单元301室,7,1995年8月18日,0,0,0,...,汉,42.248.47.174,0,NaN,0,2014.09.15-2024.09.15,2017-08-29 11:27:42.180,8,7,2017-06-10 23:40:32.000
2,F0124383-5D3A-4DDB-8E9B-F5E26DE39AF4,7a57eacfaabe48418cd40d0c84f9fc4b|699241730716368,jkand2,1.497141e+09,海南省乐东黎族自治县黄流镇镇远村三队5号,220787,198447,0,0,0,...,汉,117.136.13.204,1,1.0,1,2017.02.15-2037.02.15,2017-08-07 16:24:50.600,8,10,2017-06-10 17:24:32.791
3,DED6CF4F-60F4-477A-A57A-BDCD71250C35,1eed873d710344f4ad59f0954880cd4e|123456789001,jkioslunar,1.497194e+06,长沙市雨花区砂子塘10栋207房,14,1982年11月15日,0,0,0,...,汉,220.202.152.118,0,NaN,0,2010.10.13-2030.10.13,2017-08-09 15:54:15.950,8,7,2017-06-11 08:16:52.000
4,a087ef82-194d-44a7-96d9-00ce343b28c7,31d6bca6091d4f9c923b043eb9427f61|123456789001,jkios,1.497018e+06,江苏省东海县青湖镇北辰村2一27号,13,1996年1月24日,0,0,0,...,汉,117.136.19.125,1,2.0,2,2015.08.27-2035.08.27,2017-08-09 13:29:05.213,8,7,2017-06-09 07:25:35.000


In [8]:
from datetime import timedelta
def getWindowedData(data, apply_time, data_time, timewindow):
    p = datetime.now()
    data = data[((data[data_time]+timedelta(timewindow)) > data[apply_time])&(data[data_time]<data[apply_time])]
    print('  - filtering data using %s' % (datetime.now()-p))
    return data

In [9]:
idcard01 = getWindowedData(idcard01, 'last_decision_tm', 'ctime_stamp', 30)

  - filtering data using 0:00:00.008406


In [10]:
# 需要统计的时间
ftr_list = ['edit_address_time', 'edit_birthday_time', 'edit_id_card_number_time', 'edit_issuing_authority_time',
       'edit_name_time', 'edit_period_of_validity_time', 'edit_race_time','statistic_back_idcard_recognition_count',
       'statistic_face_recognition_count',
       'statistic_front_idcard_recognition_count']

In [11]:
replace = [x for x in ftr_list if 'statistic' in x]
for columns in replace:
    idcard01[columns] = idcard01[columns].replace('null','n',regex=True)

In [12]:
idcard01['statistic_face_recognition_count'] = idcard01['statistic_face_recognition_count'].replace('NaN',0,regex=True).fillna(0)
idcard01['statistic_face_recognition_count'] = idcard01['statistic_face_recognition_count'].astype(int)

In [13]:
# 是否所有的time也是次数
ftr = idcard01[['ugid']].drop_duplicates()
for item in ftr_list:
        temp = idcard01.groupby('ugid')[item].sum().reset_index().rename(columns = {item:'idcard_'+item})
        ftr = pd.merge(ftr, temp, how='left', on ='ugid')

In [14]:
#为什么有两次drop？
race = idcard01[['ugid', 'race', 'ctime_stamp']].drop_duplicates(['ugid', 'race'])

In [15]:
race = race.sort_values(by = ['ugid', 'ctime_stamp'], ascending = True)
race = race.drop_duplicates('ugid', keep = 'last')

In [16]:
race['race'] = race['race'].mask(race['race']!='汉', '少数民族')
race = race.drop('ctime_stamp', axis=1)
ftr = pd.merge(ftr, race, how='left', on ='ugid')

In [17]:
ftr.to_csv('./data_last/20171123_feature_idcard01_201706_201708.csv', index=False)

In [18]:
ftr

,ugid,idcard_edit_address_time,idcard_edit_birthday_time,idcard_edit_id_card_number_time,idcard_edit_issuing_authority_time,idcard_edit_name_time,idcard_edit_period_of_validity_time,idcard_edit_race_time,idcard_statistic_back_idcard_recognition_count,idcard_statistic_face_recognition_count,idcard_statistic_front_idcard_recognition_count,race
0,88beeff8-ad32-42cb-9bf7-f6fdf6448600,0,0,0,0,0,0,0,1,1,1,汉
1,98f384ff-248e-4b71-bb37-982420d98542,0,0,0,0,0,0,0,1,1,1,汉
2,cad073a5-0561-4251-bc4b-c4aa6ada1d07,0,0,0,0,0,0,0,1,1,1,汉
3,8eb7b9ec-ad72-42df-bb49-d4ff4851e13e,0,0,0,0,0,0,0,1,1,1,汉
4,3aec34f9-fa3b-43b2-9887-e649db81454c,0,0,0,0,0,0,0,1,1,1,汉
5,b50ba1b8-ed8a-44ca-a38b-f9130089313e,0,0,0,0,0,0,0,1,1,1,汉
6,b074ce4e-2993-46dd-9d09-e2e299cf0ac2,0,0,0,0,0,0,0,0,0,0,汉
7,f0393c5e-0088-4ca3-8b11-26304f80bf4f,0,0,0,0,0,0,0,1,1,1,少数民族
8,36c13d9a-c8db-4117-8703-f0beadfadbe8,0,0,0,0,0,0,0,1,1,1,汉
9,a35de5df-b2c1-4b61-ba10-067f00e1ca98,0,0,0,0,0,0,0,1,1,1,汉
